<a href="https://colab.research.google.com/github/taufiq-ai/EXAONE-3.5-2.4b-Pretrained-Finetuning-Quantization/blob/main/EXAONE_3_5_2_4B_Instruct_Pre_trained_Model_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install torch transformers huggingface_hub

In [ ]:
import warnings
import tiktoken
from bs4 import BeautifulSoup
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

from torch import bfloat16
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN") # HF_TOKEN="<Your_HuggingFace_Access_Token>" (https://huggingface.co/settings/tokens)
login(HF_TOKEN)

In [ ]:
def count_token(string: str, encoding_name: str="gpt-4o") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-4o-mini")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# If you generate HTML
def prettify_html(content):
    html = content.replace("\n"," ").replace("```", "")
    soup = BeautifulSoup(html, 'html.parser')
    pretty_html = soup.prettify()
    return pretty_html

In [ ]:
def setup_model(
    model_name_or_local_path:str="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    device="auto",
):
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_local_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_local_path,
        torch_dtype=bfloat16,
        trust_remote_code=True,
        device_map=device,
    )
    return model, tokenizer


def inference(prompt, tokenizer, model, max_tokens=200, device="cuda"):
    if type(prompt)==str:
        messages = [
            {"role": "system",
             "content": "You are a helpful e-commerce customer support chatbot."},
            {"role": "user", "content": prompt}
        ]
    else: messages=prompt

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    output = model.generate(
        input_ids.to(device),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=max_tokens,
        do_sample=False,
    )
    completion = tokenizer.decode(output[0])
    content = completion.split("[|assistant|]")[-1].split("[|endofturn|]")[0]
    return completion, content

In [ ]:
model, tokenizer = setup_model()

## Named Entity Recognition (NER)

In [ ]:
prompt_ner = """Task: Named Entity Recognition (NER)
Given a text, for NER. You have to extract name, company, location. Please return the NER in a JSON object.
Example output: {"name":"<name>", "location":"<location>", "company":"<company>", "summary":"<5-10 words summary>"}
Follow the example output format strictly, and do not write any extra speech and sign other than the expected output.
If any entity is not present, write as empty string ("").

Text: "OpenAI's board of directors fired co-founder and CEO Sam Altman on November 17, 2024. The board said it had no confidence in Altman's leadership due to concerns about his handling of AI safety and allegations of abusive behavior. The board also said that Altman was not consistently candid in his communications with them."
"""
completion, content = inference(prompt=prompt_ner, tokenizer=tokenizer, model=model)
content

## Classification

In [ ]:
prompt_cls = """TASK: chat text classification. You classify the chat text send by customers.
Baased on your classification the system will call RAG agent or skip. Classify the text into ['rag', 'skip', 'admin'] with confidence 0 - 1.
'admin'means: The user want make some change in order and an admin require to complete it.
'rag' means: to answer the query system need to retrieve info from vector db.
'skip' means: it is a general text such as greetings or slang and no need to call rag agent.
Return result as JSON object.
Example output: {"class": "admin", "confidence":{"rag":0.3, "skip":0.3, "admin":0.4}}
Follow the example output format strictly and do not write any extra speech and sign other than the expected output.
Text: "change my delivery location to B78 Silk Road, Dhaka."
"""
completion, content = inference(prompt=prompt_cls, tokenizer=tokenizer, model=model)
content


## Code Generation

In [ ]:
prompt_code = """Task: Django shell db filter code generation.
Given a NER json, you have to write python script to filter objects from a db table.
You have to filter based on the following field names [username, company, date, id, location] from the table 'Person'.
Given NER below, add filtering items to the generated code which is present on the NER, skip others.
example output: 'Person.objects.filter(username="<given_name>", company="<company>", id="<given_uuid", date="<given_datetime>")'
If a field name is not present in the NER then do not add that field in the generated python script.mandaatory.
such as given NER: '```json\n{"name": "Jorex Devin", "company": "Lexicon", id: "4546464646464htyhgtyyht"}\n```'then the output will be 'Person.objects.filter(username="Jorex Devin", company="Lexicon", id="4546464646464htyhgtyyht")';
Follow the example output format strictly and do not write any extra speech and sign other than the expected output.


Given NER: '```json\n{"name": "Hossain Shikder", "company": "Sony", "location": "Macau"}\n```'
"""
completion, content = inference(prompt=prompt_code, tokenizer=tokenizer, model=model)
content

In [ ]:
prompt_html = """Task: Html Code Generation.
Write an html table to compare gpt-4 and claude-3. Also add a concise comparison inside the table.
Please do not write any extra speech other thand the expected output.
"""
completion, content = inference(prompt=prompt_html, tokenizer=tokenizer, model=model, max_tokens=600)
content

**Snap of table view (cleaned)**

![image.png](attachment:44a70144-8b57-4a7a-93fb-d9cdec6b9d21.png)

## Q&A

In [ ]:
completion, content = inference(prompt="Who won the 2024 US Election?", tokenizer=tokenizer, model=model)
content

In [ ]:
completion, content = inference(prompt="Who is the prime minister of Bangladesh?", tokenizer=tokenizer, model=model)
content

## Summarization

In [ ]:
prompt_summary = """TASK: Text Summarization
Given a text below. Summarize the text within 10-50 words.

Text: "OpenAI CEO and co-founder Sam Altman has been reinstated to the firm’s board of directors following an outside investigation into the turmoil that led the company to abruptly fire and rehire him in November.

OpenAI said the investigation by the law firm WilmerHale concluded that Altman’s ouster had been a “consequence of a breakdown in the relationship and loss of trust” between Altman and the prior board and that the CEO’s conduct “did not mandate removal”.

OpenAI Holds Its First Developer Conference<br>SAN FRANCISCO, CALIFORNIA - NOVEMBER 06: Microsoft CEO Satya Nadella (R) speaks as OpenAI CEO Sam Altman (L) looks on during the OpenAI DevDay event on November 06, 2023 in San Francisco, California. Altman delivered the keynote address at the first ever Open AI DevDay conference. (Photo by Justin Sullivan/Getty Images)
Microsoft asks to dismiss New York Times’s ‘doomsday’ copyright lawsuit
Read more
For more than three months, OpenAI said little about what led its then board of directors to fire Altman on 17 November. In a statement at the time, the board accused Altman of not being “consistently candid in his communications”, adding that it “no longer had confidence in his ability to lead”. Less than a week later he was reinstated as CEO after many at the company threatened to resign unless he returned to his role, including board chair Greg Brockman, who responded to the ouster by quitting his job as the company’s president.

In addition to sharing the results of the investigation, the ChatGPT maker announced has added three women to its board of directors: Sue Desmond-Hellmann, a former CEO of the Bill & Melinda Gates Foundation; Nicole Seligman, a former Sony general counsel; and Instacart CEO Fidji Simo.

The actions are a way for the San Francisco-based artificial intelligence company to show investors and customers that it is trying to move past the internal conflicts that nearly destroyed it last year and made global headlines.

Much of OpenAI’s conflicts are rooted in its unusual governance structure. Founded as a non-profit with a mission to safely build futuristic AI that helps humanity, it is now a fast-growing big business still controlled by a non-profit board bound to its original mission.

An investigation released by the New York Times on Thursday shed light on conditions within the company leading up to Altman’s ousting, stating that OpenAI’s chief technology officer Mira Murati had called into question the CEO’s management style. OpenAI co-founder and chief scientist Ilya Sutskever reportedly raised similar concerns about Altman’s alleged “history of manipulative behavior”.

Altman alluded to such allegations in a call with reporters on Friday, stating that he had been disheartened to see people leaking information to try to “pit us against each other” and demoralize the team. At the same time, he said he had learned from the experience and apologized for a dispute with a former board member he could have handled “with more grace and care”.

“I’m pleased this whole thing is over,” he said.

The investigation found that the prior board had acted within its discretion. But it also determined that Altman’s “conduct did not mandate removal”, OpenAI said. It said both Altman and Brockman remained the right leaders for the company.

“The review concluded there was a significant breakdown in trust between the prior board, and Sam and Greg,” Bret Taylor, the board’s chair, told reporters on Friday. “And similarly concluded that the board acted in good faith, that the board believed at the time that actions would mitigate some of the challenges that it perceived and didn’t anticipate some of the instability.”

Days after his surprise ouster, Altman and his supporters – with backing from most of OpenAI’s workforce and close business partner Microsoft – helped orchestrate a comeback that brought Altman and Brockman back to their executive roles and forced out board members Tasha McCauley, Helen Toner and Ilya Sutskever, though the latter kept his job as chief scientist.

Altman and Brockman did not regain their board seats at that time. But an “initial” new board of three men was formed, led by Taylor, a former Salesforce and Facebook executive who also chaired Twitter’s board before Elon Musk took over the platform. The others were the former US treasury secretary Larry Summers and the Quora CEO, Adam D’Angelo, the only member of the previous board to stay on.

OpenAI had retained the law firm WilmerHale to investigate the events that led to Altman’s ouster. During the investigation, OpenAI said, WilmerHale conducted dozens of interviews with the company’s previous board, current executives, advisers and other witnesses. The company also said the law firm reviewed thousands of documents and other corporate actions.

The board said it will also be making “improvements” to the company’s governance structure. It said it will adopt new corporate governance guidelines, strengthen the company’s policies around conflicts of interest, create a whistleblower hotline that will allow employees and contractors to submit anonymous reports and establish additional board committees.

The company still has other troubles to contend with, including a lawsuit filed by the billionaire Elon Musk, who helped bankroll the early years of OpenAI and was a co-chair of its board after its 2015 founding. Musk alleges that the company is betraying its founding mission in pursuit of profits.

Legal experts have expressed doubt about whether Musk’s arguments, centered on an alleged breach of contract, will hold up in court.

But it has already forced open the company’s internal conflicts about its unusual governance structure, how “open” it should be about its research and how to pursue what’s known as artificial general intelligence, or AI systems that can perform just as well as – or even better than – humans in a wide variety of tasks.

OpenAI and Microsoft have also been sued by a range of news outlets, including the New York Times, the Intercept, AlertNet and Raw Story over allegations that their generative artificial intelligence products violated copyright laws."
"""

completion, content = inference(prompt=prompt_summary, tokenizer=tokenizer, model=model)
content

## Text Generation

In [ ]:
prompt_title = """Task: Title Generation. Given few keywords, generate 5 titles based on these and return as a list inside JSON.
example output: {"titles":[<5 titles>]}
Keywords: ['gardening', 'asparagas']
"""
completion, content = inference(prompt=prompt_title, tokenizer=tokenizer, model=model)
content

In [ ]:
prompt_article = """Task: Long Blog Article Generation.

Given a title. Write a large blog article with 1000 words based on the title.
Add multiple heading and subheadings.
Feel free to add multiple paragraphs, list, tables, quotes in the content.

Format the article with Markdown.

title: 'Grow Your Green Thumb: Asparagus Harvest Guide'
"""

completion, content = inference(
    prompt=prompt_article,
    max_tokens=1500,
    tokenizer=tokenizer,
    model=model
)
print(f"generated tokens: {count_token(content)}, generated words: {len(content.split())}")

In [ ]:
content

👉 **The Output has been pretified using Claude.AI**

# Grow Your Green Thumb: Asparagus Harvest Guide

Asparagus, often hailed as a culinary delicacy and a symbol of springtime freshness, offers gardeners and home cooks alike a delightful experience when grown properly. Whether you're a seasoned gardener or just starting out, understanding when and how to harvest asparagus can significantly enhance your yield and enjoyment of this versatile vegetable. This comprehensive guide will walk you through everything you need to know about harvesting asparagus effectively, ensuring you enjoy plump, tender spears right from your garden.

## Why Harvesting Timing Matters

The timing of asparagus harvesting is crucial for both flavor and yield. Asparagus plants typically take about 18 to 24 weeks from planting to maturity, depending on the variety and growing conditions. Here's why timing is so important:

### Optimal Harvest Window

- **Early Season**: Asparagus harvested early in the season tends to be tender and crisp, ideal for spring dishes.
- **Late Season**: Harvesting later can result in tougher spears due to increased maturity and potential exposure to cooler temperatures.

### Seasonal Considerations

- **Spring**: Ideal for fresh spring meals.
- **Summer**: Some varieties can be harvested through summer, though they may become less tender.
- **Fall**: Harvesting in fall can extend your asparagus season into cooler months.

## Preparing Your Garden for Asparagus

Before diving into harvesting, ensuring your garden is set up for optimal asparagus growth is key:

### Soil Preparation

- **Well-Drained Soil**: Asparagus prefers loose, well-drained soil rich in organic matter. Amend your soil with compost to improve fertility and drainage.
- **pH Level**: Aim for a slightly acidic to neutral pH range (6.0 to 7.0).

### Planting Tips

- **Spacing**: Space asparagus plants about 12 to 18 inches apart to allow for proper air circulation and growth.
- **Depth**: Plant seedlings at the same depth they were in their containers.

### Watering and Maintenance

- **Regular Watering**: Keep the soil consistently moist but not waterlogged. Asparagus thrives in environments with moderate moisture.
- **Mulching**: Apply a layer of mulch around plants to retain moisture and suppress weeds.

## Harvesting Techniques

Mastering the art of harvesting asparagus not only ensures you get the best quality but also promotes healthy plant growth for future harvests. Here's how to do it right:

### Timing Your Harvest

- **Early Signs**: Look for young shoots that are about 6 inches tall with leaves that are just beginning to unfurl.
- **Harvesting Tools**: Use sharp, clean scissors or pruning shears to avoid damaging the plant.

### Harvesting Process

1. **Identify the Shoots**: Locate the young shoots that are tender and free from any signs of damage or disease.
2. **Cutting Technique**: Cut the shoots just above the soil level using your tools. Aim for a clean cut to minimize stress on the plant.
3. **Quantity**: Harvest about 4-6 inches of asparagus per plant per season to allow room for new growth.

### Post-Harvest Care

- **Immediate Use**: Enjoy freshly harvested asparagus within a day or two for optimal flavor and texture.
- **Storage**: If not using immediately, store harvested asparagus in a cool place wrapped in damp paper towels or placed in a sealed container lined with paper towels to absorb excess moisture.

## Tips for Maximizing Yield

To ensure a bountiful asparagus harvest, consider these additional tips:

### Rotational Planting

- **Succession Planting**: Plant new asparagus beds every spring to extend your harvest period.
- **Companion Planting**: Pair asparagus with plants like carrots or onions, which can help deter pests naturally.

### Disease Management

- **Regular Inspection**: Keep an eye out for signs of pests like aphids or diseases like asparagus root rot.
- **Sanitation**: Remove dead plant material promptly to prevent disease spread.

### Nutrient Boosting

- **Fertilizers**: Apply balanced fertilizers during the growing season to support robust growth.
- **Organic Options**: Consider using compost teas or organic fertilizers rich in nitrogen to promote healthy foliage and spears.

## Culinary Uses of Fresh Asparagus

Fresh asparagus adds a vibrant touch to numerous dishes, enhancing both flavor and presentation:

### Classic Dishes

- **Asparagus Salad**: Combine fresh asparagus with cherry tomatoes, cucumbers, and a light vinaigrette.
- **Grilled Asparagus**: Grill asparagus spears until tender and slightly charred for a smoky flavor.

### Creative Applications

- **Asparagus Risotto**: Incorporate asparagus into creamy risotto for a hearty winter meal.
- **Asparagus Stir-Fry**: Quick stir-fry with garlic, ginger, and soy sauce for a quick weeknight dinner.

## Conclusion

Harvesting asparagus correctly is not just about getting the best flavor; it's about nurturing your garden sustainably and enjoying the fruits of your labor year after year. By paying attention to timing, soil health, and proper harvesting techniques, you can ensure a plentiful and delicious asparagus harvest. Whether you're a seasoned gardener or just starting out, these insights will guide you toward mastering the art of growing and harvesting asparagus, enriching both your culinary experiences and garden health.

---

### Additional Resources

- **Books**: "The Vegetable Gardener's Bible" by Edward C. Smith
- **Websites**: USDA Natural Resources Conservation Service (NRCS) for detailed gardening advice
- **Apps**: Gardening apps like Gardenely offer tips and reminders for planting and harvesting schedules

Enjoy exploring the vibrant world of asparagus cultivation and savor the rewards of your green thumb!